In [1]:
import pandas as pd
import numpy as np

import psycopg2
redshift_endpoint = "boostdatalake.c7h2xtvjdpxd.ap-southeast-1.redshift.amazonaws.com"
redshift_user = "fabian"
redshift_pass = "Gtd87jRe"
port = 5439
dbname = "datalake"

In [2]:
from sqlalchemy import create_engine
from sqlalchemy import text
engine_string = "postgresql+psycopg2://%s:%s@%s:%d/%s" \
% (redshift_user, redshift_pass, redshift_endpoint, port, dbname)
engine = create_engine(engine_string)

In [17]:
sql = """
select
identifier, registeredname, merchantcategory, a.vtm_accountid, a.vtm_skucode, a.vtm_additionaldata_outletid, first_dt,
sum(gtv) as gtv, count(distinct vac_customerid) as users, count(dt) as txn
from
(
  select vtm_accountid, vtm_skucode, vtm_additionaldata_outletid, abs(vtm_amount)/100.0 as gtv, vac_customerid, vtc_datecreated as dt 
  from aggregation_layer.transaction_customer_merchant
  where dt >= '2021-01-01'
  and vtm_type = 'PAYMENT' and vtc_category = 'PAYMENT'
) as a

inner join (select * from csv_files.merchantlist_mongo where identifier in ('MCM0087953', 'MCM0087956', 'MCM0109085')) as b
on a.vtm_accountid = b.merchantaccountid

left join (
  select vtm_accountid, vtm_skucode, vtm_additionaldata_outletid, min(vtc_datecreated) as first_dt from aggregation_layer.transaction_customer_merchant
  group by 1, 2, 3
  having first_dt >= '2021-01-01'
) as c
on a.vtm_accountid = c.vtm_accountid
and a.vtm_skucode = c.vtm_skucode
and a.vtm_additionaldata_outletid = c.vtm_additionaldata_outletid

group by 1, 2, 3, 4, 5, 6, 7
order by 1, 5, 6
"""

In [7]:
sql = """
--select distinct registeredname, mcmid, merchantaccountid, skucode, additionaldata_outletid, partner_, vertical
--from public.online_table_new where dt >= '2020-01-01'
select * from csv_files.merchantlist_mongo
"""

In [8]:
sql = """
select
distinct identifier, registeredname, merchantcategory, a.vtm_accountid, a.vtm_skucode, a.vtm_additionaldata_outletid
from
(
  select vtm_accountid, vtm_skucode, vtm_additionaldata_outletid
  from aggregation_layer.transaction_customer_merchant
  
) as a

inner join 
(select * from csv_files.merchantlist_mongo where identifier in ('MCM0115835') -- ala carte 
) as b
on a.vtm_accountid = b.merchantaccountid
order by 1, 5, 6
"""

In [8]:
df = pd.read_sql_query(text(sql), engine)

In [9]:
df.head()

,identifier,merchantcategory,businessprofileid,merchantaccountid,merchantid,type,registeredname
0,MCM0008842,RETAIL,5cf0787fb5618f4208d31400,5cf08ad4b5618f4208d530d5,5cf083dab5618f4208d3ca53,COMPANY,BUYTICKETS
1,MCM0008870,RETAIL,5cf0787fb5618f4208d3141c,5cf08ad4b5618f4208d530f1,5cf083dab5618f4208d3ca5e,COMPANY,Nardell Sdn Bhd
2,MCM0008898,RETAIL,5cf0787fb5618f4208d31438,5cf08ad4b5618f4208d5310d,5cf083dab5618f4208d3ca69,COMPANY,PD Beach Coconut Milkshake
3,MCM0008926,INDIVIDUAL,5cf0787fb5618f4208d31454,5cf08ad4b5618f4208d53129,5cf08446b5618f4208d43032,INDIVIDUAL,
4,MCM0008954,INDIVIDUAL,5cf0787fb5618f4208d31470,5cf08ad4b5618f4208d53145,5cf08446b5618f4208d43049,INDIVIDUAL,


In [10]:
df.to_excel('20210722_MerchantList_Mongo.xlsx')